Наши данные

In [1]:
import pandas as pd
import re

pth = '/home/sondors/Documents/price/BERT_NER/csv/Kazakov_241123/Continental_character.xlsx'   
df = pd.read_excel(pth, dtype=str)
df
# print(df.keys())

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Комментарии,Наименование,Модель,Ширина шины,Высота профиля,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34
0,96794,NaN,8101,NaN,7317,NaN,[CONTISPORTCONTACT 5 SUV 23555R19 105W XL FR L...,SportContact 5 SUV,235,55,...,NaN,NaN,Auto,Все товары :: Авто :: Шины и диски :: Шины,Continental,https://avatars.mds.yandex.net/get-mpic/527704...,[8101],2023-07-29 18:50:34,NaN,1768586022
1,61852,price,8101,NaN,7317,NaN,125/80 R13 Continental ContiWinterContact TS80...,WinterContact TS800,125,80,...,355286,NaN,Auto,Шины,Continental,https://euro-diski.ru/upload/iblock/dd6/a72e54...,"[8101, 1403, 350201]",2023-08-09 14:10:08,NaN,1777012756
2,96794,NaN,8101,NaN,7317,NaN,14/155/65 Continental IceContact 3 TA 75T,IceContact 3,155,65,...,NaN,NaN,Auto,Все товары :: Авто :: Шины и диски :: Шины,Continental,https://avatars.mds.yandex.net/get-mpic/449302...,[8101],2023-07-29 18:50:34,NaN,1754590109
3,61852,NaN,8101,NaN,7317,NaN,14/185/60 Continental IceContact 3 TA 82T,IceContact 3,185,60,...,347594,NaN,Auto,Шины,Continental,https://euro-diski.ru/upload/iblock/72a/cf125c...,[8101],2023-07-07 14:50:12,NaN,1755712962
4,96794,NaN,8101,NaN,7317,NaN,145/65 R15 72T Continental VikingContact 7,VikingContact 7,145,65,...,NaN,NaN,Auto,Все товары :: Авто :: Шины и диски :: Шины,Continental,https://avatars.mds.yandex.net/get-mpic/136080...,"[8101, 829805]",2023-05-05 05:06:00,NaN,1694032869
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20276,96794,NaN,8101,NaN,7317,NaN,Континенталь 295/30/19 W 100 ContiWinterContac...,WinterContact TS830,295,30,...,NaN,NaN,Auto,Все товары :: Авто :: Шины и диски :: Шины,Continental,https://avatars.mds.yandex.net/get-mpic/408043...,"[8101, 2110, 7502, 5999]",2023-05-02 11:20:06,NaN,1694349152
20277,96794,NaN,8101,NaN,7317,NaN,Континенталь 295/30/21 V 102 WinterContact TS ...,WinterContact TS860,295,30,...,NaN,NaN,Auto,Все товары :: Авто :: Шины и диски :: Шины,Continental,https://avatars.mds.yandex.net/get-mpic/765032...,"[8101, 7502]",2023-05-02 17:08:57,NaN,1695507907
20278,96794,NaN,8101,NaN,7317,NaN,Континенталь 295/40/19 V 108 ContiWinterContac...,WinterContact TS830,295,40,...,NaN,NaN,Auto,Все товары :: Авто :: Шины и диски :: Шины,Continental,https://avatars.mds.yandex.net/get-mpic/408043...,[8101],2023-06-08 16:53:29,NaN,1694665532
20279,96794,NaN,8101,NaN,7317,NaN,Континенталь 305/40/20 Y 112 ContiSportContact...,SportContact 5,305,40,...,NaN,NaN,Auto,Все товары :: Авто :: Шины и диски :: Шины,Continental,https://avatars.mds.yandex.net/get-mpic/405568...,"[8101, 8398, 719825]",2023-05-02 17:08:57,NaN,1695320371


In [2]:
def process_text(input_text):
    def separate_letters_and_numbers(input_text):
        # Используем регулярное выражение для поиска сочетаний букв и цифр
        pattern = re.compile(r'(\D+|\d+)')
        
        # Используем findall для нахождения всех сочетаний
        matches = pattern.findall(input_text)
        # Возвращаем строку с пробелами между буквами и цифрами
        return ' '.join(matches)

    processed_text = separate_letters_and_numbers(input_text)    
    processed_text = processed_text.replace("|", " | ")
    processed_text = processed_text.replace("(", " ( ")
    processed_text = processed_text.replace(")", " ) ")
    processed_text = processed_text.replace("[", " [ ")
    processed_text = processed_text.replace("]", " ] ")
    # Убираем повторяющиеся пробелы
    processed_text = re.sub(r'  +', ' ', processed_text)
    return processed_text

def create_bio_tags(row):
    def remove_word(line, word):
        # Преобразуем обе строки в нижний регистр для сравнения без учета регистра
        lower_line = line.lower()
        lower_word = word.lower()

        # Проверяем, содержится ли подстрока в строке
        if lower_word in lower_line:
            # Используем регулярное выражение для удаления подстроки (с учетом регистра)
            line = re.sub(fr'\b{re.escape(word)}\b', '', line, flags=re.IGNORECASE).strip()

        return line
        
    name = str(row['offer'])
    width = str(row['width'])
    height = str(row['height'])
    radius = str(row['radius'])
    brand = row['brand']
    line = str(row['line'])
    v_ind = str(row['v_ind'])

    # Создаем пустой список для хранения тегов
    tags = ['O'] * len(name.split())
    Begin_line = True
    Begin_v_ind = True
    Begin_radius = True
    # Проверяем каждое слово в наименовании и устанавливаем соответствующий тег
    for i, word in enumerate(name.split()):
        if word == '/':
            continue
        
        #   width
        if word == width:
            tags[i] = 'B-width'
            width = width.replace(word,"")

        #   height
        elif word == height:
            tags[i] = 'B-height'
            height = height.replace(word,"")

        #   radius    
        elif word.upper() in radius.upper():
            if radius.replace(" ","") == "":
                continue

            if Begin_radius:
                tags[i] = 'B-radius'
                Begin_radius = False
            else:
                tags[i] = 'I-radius'
            radius = radius.replace(word.lower(),"").replace(word.upper(),"")

        #   brand
        elif word in brand:
            tags[i] = 'B-brand'

        #   v_ind
        elif word.upper() in v_ind.upper():
            if v_ind.replace(" ","") == "":
                continue

            if Begin_v_ind:
                tags[i] = 'B-v_ind'
                Begin_v_ind = False
            else:
                tags[i] = 'I-v_ind'  
            v_ind = v_ind.replace(word.lower(),"").replace(word.upper(),"")  

        #   line
        elif word.upper() in line.upper() or re.sub(r"[^a-zA-Zа-яА-Я]","", word.upper().replace("СПОРТ", "SPORT")) in line.upper():
            if len(re.sub(r"[^a-zA-Zа-яА-Я]","", word.upper().replace("СПОРТ", "SPORT")))<3 and Begin_line:
                continue
            if line.replace(" ","") == "":
                continue
            
            if Begin_line:
                tags[i] = 'B-line'
                Begin_line = False
            else:
                tags[i] = 'I-line'
            line = min(remove_word(line, word), line.replace(word,""))

        elif word.startswith('Conti') and word[5:] in line:    #   для Continental добавим искусственно
            if line.replace(" ","") == "":
                continue
            
            if Begin_line:
                tags[i] = 'B-line'
                Begin_line = False
            else:
                tags[i] = 'I-line'
            line = min(remove_word(line, word), line.replace(word,""))

    # Собираем теги в строку
    return ','.join(tags)

def change_columns(df):   
    new_df = pd.DataFrame()

    df = df.rename(columns={'Наименование': 'offer'})
    df = df[df['offer'].notna()]

    new_df['offer'] = df['offer']
    new_df['width'] = df['Ширина шины']
    new_df['height'] = df['Высота профиля']
    new_df['radius'] = df['Диаметр']
    new_df['v_ind'] = df['Индекс нагрузки и скорости']
    new_df['brand'] = df['Unnamed: 29']
    new_df['line'] = df['Модель']

    def process_column(row, column_name):
        # Проверка, является ли значение в "offer" подстрокой значения из указанной колонки
        contains_column_value = str(row['offer']).find(str(row[column_name])) != -1
        
        # Если значение из колонки не найдено в "offer"
        if not contains_column_value:
            # Замена "." на "," в соответствующей строке из columns_to_process
            row[column_name] = str(row[column_name]).replace('.', ',')
            
            # Повторная проверка
            contains_column_value = str(row['offer']).find(str(row[column_name])) != -1

            # Если проверка не пройдена, удаляем строку из таблицы
            if not contains_column_value:
                return None
        return row
    
    def process_radius1(txt):
        return txt.replace("r", "").replace("R", "")
    
    new_df['radius'] = new_df['radius'].apply(process_radius1)
    columns_to_process = ['width', 'height', 'radius']
    for column in columns_to_process:
        new_df = new_df.apply(process_column, args=(column,), axis=1)

    # Удаляем строки, в которых проверка не пройдена
    new_df = new_df.dropna().reset_index(drop=True)

    def process_radius(txt):
        return "R"+txt
    
    new_df['radius'] = new_df['radius'].apply(process_radius)
    new_df['offer'] = new_df['offer'].apply(process_text)
    new_df['brand'] = new_df['offer'].apply(extract_Continental)
    new_df['BIO_Tags'] = new_df.apply(create_bio_tags, axis=1)

    keywords = ['width', 'height', 'radius', 'v_ind', 'brand', 'line']

    # Проверяем вхождение каждого ключевого слова в каждую строку
    for keyword in keywords:
        new_df = new_df[new_df['BIO_Tags'].str.contains(keyword, case=False)]

    # Переиндексируем DataFrame после удаления строк
    new_df = new_df.reset_index(drop=True)

    new_df = new_df[new_df['v_ind'].apply(lambda x: len(str(x)) <= 5)]
    # Переиндексация DataFrame после удаления строк
    new_df = new_df.reset_index(drop=True)

    return new_df

def extract_Continental(text):
    brands = re.findall(r'Continental|CONTINENTAL|Континенталь|continental|КОНТИНЕНТАЛЬ', text)
    return brands

new_df = change_columns(df)
new_df

,offer,width,height,radius,v_ind,brand,line,BIO_Tags
0,125 / 80 R 13 Continental ContiWinterContact T...,125,80,R13,65Q,[Continental],WinterContact TS800,"B-width,O,B-height,B-radius,I-radius,B-brand,B..."
1,14 / 155 / 65 Continental IceContact 3 TA 75 T,155,65,R14,75T,[Continental],IceContact 3,"B-radius,O,B-width,O,B-height,B-brand,B-line,I..."
2,14 / 185 / 60 Continental IceContact 3 TA 82 T,185,60,R14,82T,[Continental],IceContact 3,"B-radius,O,B-width,O,B-height,B-brand,B-line,I..."
3,145 / 65 R 15 72 T Continental VikingContact 7,145,65,R15,72T,[Continental],VikingContact 7,"B-width,O,B-height,B-radius,I-radius,B-v_ind,I..."
4,145 / 65 R 15 Continental ContiVikingContact 6...,145,65,R15,72T,[Continental],VikingContact 6,"B-width,O,B-height,B-radius,I-radius,B-brand,B..."
...,...,...,...,...,...,...,...,...
19314,Континенталь 295 / 30 / 19 W 100 ContiWinterCo...,295,30,R19,100W,[Континенталь],WinterContact TS830,"B-brand,B-width,O,B-height,O,B-radius,B-v_ind,..."
19315,Континенталь 295 / 30 / 21 V 102 WinterContact...,295,30,R21,102V,[Континенталь],WinterContact TS860,"B-brand,B-width,O,B-height,O,B-radius,B-v_ind,..."
19316,Континенталь 295 / 40 / 19 V 108 ContiWinterCo...,295,40,R19,108V,[Континенталь],WinterContact TS830,"B-brand,B-width,O,B-height,O,B-radius,B-v_ind,..."
19317,Континенталь 305 / 40 / 20 Y 112 ContiSportCon...,305,40,R20,112Y,[Континенталь],SportContact 5,"B-brand,B-width,O,B-height,O,B-radius,B-v_ind,..."


In [3]:
new_df.to_csv('/home/sondors/Documents/price/BERT_NER/csv/Kazakov_241123/разметка/Continental.csv', sep=';',index=False)

In [4]:
index = 16389
example = dict(new_df.iloc[index])
print(example['offer'],'\n')
offer = example['offer'].split()
BIO_Tags = example['BIO_Tags'].split(',')

for i in range(len(offer)):
    print(f"{offer[i]}: {BIO_Tags[i]}")

Continental VikingContact 7 195 / 55 R 16 91 T 

Continental: B-brand
VikingContact: B-line
7: I-line
195: B-width
/: O
55: B-height
R: B-radius
16: I-radius
91: B-v_ind
T: I-v_ind


In [5]:
example

{'offer': 'Continental VikingContact 7 195 / 55 R 16 91 T',
 'width': '195',
 'height': '55',
 'radius': 'R16',
 'v_ind': '91T',
 'brand': ['Continental'],
 'line': 'VikingContact 7',
 'BIO_Tags': 'B-brand,B-line,I-line,B-width,O,B-height,B-radius,I-radius,B-v_ind,I-v_ind'}

In [6]:
example1 = {'offer': 'Континенталь 305 / 40 / 20 Y 112 SportContact SportContact 5 SportContact 55 555 112 S 40 1234567890fsfdh',
 'width': '305',
 'height': '40',
 'radius': 'R20',
 'v_ind': '112Y',
 'brand': ['Континенталь'],
 'line': 'SportContact 5'}

example1_BIO = create_bio_tags(example1)

print(example1['offer'],'\n')
offer = example1['offer'].split()
BIO_Tags = example1_BIO.split(',')

for i in range(len(offer)):
    print(f"{offer[i]}: {BIO_Tags[i]}")

Континенталь 305 / 40 / 20 Y 112 SportContact SportContact 5 SportContact 55 555 112 S 40 1234567890fsfdh 

Континенталь: B-brand
305: B-width
/: O
40: B-height
/: O
20: B-radius
Y: B-v_ind
112: I-v_ind
SportContact: B-line
SportContact: O
5: I-line
SportContact: O
55: O
555: O
112: O
S: O
40: O
1234567890fsfdh: O


In [7]:
label2id = {'B-width': 1,
            'B-height': 2, 
            'B-radius': 3, 
            'I-radius': 4,
            'B-brand': 5, 
            'B-line': 6, 
            'I-line': 7,
            'B-v_ind': 8,
            'I-v_ind': 9,
            'O': 0}
    
id2label = {1: 'B-width',
            2: 'B-height', 
            3: 'B-radius', 
            4: 'I-radius',
            5: 'B-brand', 
            6: 'B-line', 
            7: 'I-line',
            8: 'B-v_ind',
            9: 'I-v_ind',
            0: 'O'}